**INSTALLATION**

In [ ]:
!git clone --recurse-submodules https://github.com/safednn-nasa/ProphecyPlus

fatal: destination path 'ProphecyPlus' already exists and is not an empty directory.


In [ ]:
%cd ./ProphecyPlus
!pwd
!ls -lt dataset_models/airfoil_self_noise/

/content/ProphecyPlus
/content/ProphecyPlus
total 15096
-rw-r--r-- 1 root root   940648 Jan  1 02:10 mnist_cnn.h5
-rw-r--r-- 1 root root       38 Jan  1 02:10 readme.txt
-rw-r--r-- 1 root root 14250974 Jan  1 02:10 mnist.zip
-rw-r--r-- 1 root root   237800 Jan  1 02:10 cnn_max_mnist2.h5
-rw-r--r-- 1 root root    12850 Jan  1 02:10 inp_arr_acts.csv


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 29.1 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing 

**LOAD DATA**

In [ ]:
import numpy as np
data = np.genfromtxt("/content/ProphecyPlus/dataset_models/airfoil_self_noise/airfoil_self_noise.csv", delimiter=',')
print(data.shape)

inputstr = data[:,0:5]
outputstr = data[:,5]
print(inputstr.shape)
print(inputstr[0])

inputs = []
outputs = []
for indx in range(0, len(inputstr)):
  input = []
  for indx1 in range(0, len(inputstr[indx])):
    val = np.float32(inputstr[indx][indx1])
    input.append(val)
    if (indx == 0):
      print(type(val), val)
      print(input)
  inputs.append(input)
  outputs.append(np.float32(outputstr[indx]))

print(inputs[0])
print(outputs[0])

def z_score_normalize(data):
    """
    Applies z-score normalization (subtracts mean, divides by standard deviation) to the input data.

    Args:
        data (numpy.ndarray): The data to be normalized. Each row is a sample, and each column is a feature.

    Returns:
        normalized_data (numpy.ndarray): The normalized data.
        mean (numpy.ndarray): The mean of the original data (per feature).
        std (numpy.ndarray): The standard deviation of the original data (per feature).
    """
    # Convert data to NumPy array if it's a list
    data = np.asarray(data)

    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)

    # Avoid division by zero
    # Check if std is a scalar (for 1D data)
    if np.isscalar(std):
        if std == 0:
            std = 1
    else:  # For 2D data (or higher)
        std[std == 0] = 1

    normalized_data = (data - mean) / std
    return normalized_data, mean, std

(normalized_data, mean, std)= z_score_normalize(inputs)
print(normalized_data[0])
(normalized_output, mean, std)= z_score_normalize(outputs)
print(normalized_output[0])


(1503, 6)
(1503, 5)
[8.00000e+02 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]
<class 'numpy.float32'> 800.0
[800.0]
<class 'numpy.float32'> 0.0
[800.0, 0.0]
<class 'numpy.float32'> 0.3048
[800.0, 0.0, 0.3048]
<class 'numpy.float32'> 71.3
[800.0, 0.0, 0.3048, 71.3]
<class 'numpy.float32'> 0.00266337
[800.0, 0.0, 0.3048, 71.3, 0.00266337]
[800.0, 0.0, 0.3048, 71.3, 0.00266337]
126.201
[-0.6620218 -1.1463989  1.7992973  1.312917  -0.6448043]
0.1979379


**LOAD ONNX AND H5 MODELS**

In [ ]:
! pip install onnx
! pip install onnx-tf

! pip install pybind11
! pip install cmake

import torch


import onnx
!pip install onnxruntime
import onnxruntime

In [ ]:
session = onnxruntime.InferenceSession("/content/ProphecyPlus/dataset_models/airfoil_self_noise/renamed_model.onnx", None)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print(input_name)
print(output_name)


results = []
for indx in range(0,len(normalized_data)):
  input_data = (normalized_data[indx]).reshape(1, -1).astype(np.float32)
  result = session.run([output_name], {input_name: input_data})
  results.append(result[0][0][0])



from sklearn.metrics import r2_score
print(np.shape(results), np.shape(outputs))
r2 = r2_score(results,normalized_output)
print("R2 Score:", r2)

layer_0_input_0
layer_6_output_0
(1503,) (1503,)
R2 Score: 0.8916182518005371


In [ ]:
import numpy as np

import tensorflow as tf
import keras


print('Loading the model:')
model_h5=tf.keras.models.load_model("/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5")
print("Printing summary of the model:")
model_h5.summary()

    # Get predictions
results_h5_train = model_h5.predict(normalized_data_train)
print(np.shape(results_h5_train), np.shape(normalized_output_train))
results_h5_train_flat = results_h5_train.flatten()
print(np.shape(results_h5_train_flat))
print(np.shape(normalized_output_train))
r2 = r2_score(results_h5_train_flat,normalized_output_train)
print("H5 R2 Score:", r2)

mean_abs_err = np.mean(np.abs(results_h5_train_flat - normalized_output_train))
print("H5 Mean Absolute Error:", mean_abs_err)

Loading the model:
Printing summary of the model:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_0_input_0 (InputLaye  [(None, 5)]               0         
 r)                                                              
                                                                 
 layer_0_output_0 (Dense)    (None, 256)               1536      
                                                                 
 layer_1_output_0 (Activati  (None, 256)               0         
 on)                                                             
                                                                 
 layer_2_output_0 (Dense)    (None, 256)               65792     
                                                                 
 layer_3_output_0 (Activati  (None, 256)               0         
 on)                                                             
           

**LABEL REGRESSION OUTPUTS, 0 ( Y-YIDEAL > 0.192 (MAE)) ELSE 1)**

In [ ]:
# LABEL INPUTS 0 ( Y-YIDEAL > 1.0854 ELSE 1)
labels_train = []
op_vals_cor = []
op_vals_inc = []
for indx in range(0,len(normalized_data_train)):
  if (np.abs(normalized_output_train[indx] - results_h5_train_flat[indx]) >= 0.19200696):
    labels_train.append(0)
    op_vals_inc.append(results_h5_train_flat[indx])
  else:
    labels_train.append(1)
    op_vals_cor.append(results_h5_train_flat[indx])

print("TR COUNT 1:", labels_train.count(1))
print("TR COUNT 0:", labels_train.count(0))

print("MEAN CORRECT OP VAL:", np.mean(op_vals_cor))
print("MAX CORRECT OP VAL:", np.max(op_vals_cor))
print("MIN CORRECT OP VAL:", np.min(op_vals_cor))
std_dev_cor = np.std(op_vals_cor)
print("STD CORRECT OP VAL:", std_dev_cor)

print("MEAN INCORRECT OP VAL:", np.mean(op_vals_inc))
print("MAX INCORRECT OP VAL:", np.max(op_vals_inc))
print("MIN INCORRECT OP VAL:", np.min(op_vals_inc))
std_dev_inc = np.std(op_vals_inc)
print("STD CORRECT OP VAL:", std_dev_inc)



results_h5_test = model_h5.predict(normalized_data_test)
results_h5_test_flat = results_h5_test.flatten()


labels_test = []
for indx in range(0,len(normalized_data_test)):
  if (np.abs(normalized_output_test[indx] - results_h5_test_flat[indx]) >= 0.19200696):
    labels_test.append(0)
  else:
    labels_test.append(1)

print("TEST COUNT 1:", labels_test.count(1))
print("TEST COUNT 0:", labels_test.count(0))



TR COUNT 1: 788
TR COUNT 0: 414
MEAN CORRECT OP VAL: -0.08674386
MAX CORRECT OP VAL: 1.9108756
MIN CORRECT OP VAL: -3.1071632
STD CORRECT OP VAL: 0.94361866
MEAN INCORRECT OP VAL: 0.16865167
MAX INCORRECT OP VAL: 1.9617213
MIN INCORRECT OP VAL: -2.936437
STD CORRECT OP VAL: 0.93601966
10/10 [==============================] - 0s 2ms/step
TEST COUNT 1: 171
TEST COUNT 0: 130


In [ ]:
np.save('./x_train_npy.npy',normalized_data_train)
np.save('./y_train_npy.npy',labels_train)
np.save('./x_test_npy.npy',normalized_data_test)
np.save('./y_test_npy.npy',labels_test)

**Rules from layer 5 based on (on/off) activations**

In [ ]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_5_output_0' -type 3 -acts True

CONFIG PARAMS: LAYER NAME: layer_5_output_0 ,TYPE: 3 ,INP TYPE: 0 ,ACTS: True ,Top/All: False
Layer Name: layer_5_output_0
Layers to be considered for fingerprinting: ['layer_5_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_5_output_0 layer
Fingerprint after layer_5_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_5_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for training set: 100% 297/297 [00:00<00:00, 1315425.86it/s]
InV 0
impure:
[[155, 97, 39, 80, 129, 7, 130, 164, 209, 160, 196, 152, 123], [0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0], -1]

Fingerprinting VAL data after layer_5_output_0 layer
Fingerprint after layer_5_output_0. ((301, 256) inputs, (301, 256) neurons)
PRINTING ALL RULES.
impure:
[[155, 97, 39, 80, 129, 7, 130, 164, 209, 160, 196, 152, 123], [0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0], -

In [ ]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]


df_op.to_csv("./rules_5_1.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_5_0.csv",index=False)


****** RULES ********


**Rules from layer 4 based on neuron values**

In [ ]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_4_output_0' -type 3

CONFIG PARAMS: LAYER NAME: layer_4_output_0 ,TYPE: 3 ,INP TYPE: 0 ,ACTS: False ,Top/All: False
Layer Name: layer_4_output_0
Layers to be considered for fingerprinting: ['layer_4_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_4_output_0 layer
Fingerprint after layer_4_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_4_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for validation set: 100% 139/139 [00:00<00:00, 1067780.69it/s]
InV 0

Fingerprinting VAL data after layer_4_output_0 layer
Fingerprint after layer_4_output_0. ((301, 256) inputs, (301, 256) neurons)
PRINTING ALL RULES.


In [ ]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]
df_op.to_csv("./rules_4_1.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]
df_op.to_csv("./rules_4_0.csv",index=False)

****** RULES ********


In [ ]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_3_output_0' -type 3 -acts True

CONFIG PARAMS: LAYER NAME: layer_3_output_0 ,TYPE: 3 ,INP TYPE: 0 ,ACTS: True ,Top/All: False
Layer Name: layer_3_output_0
Layers to be considered for fingerprinting: ['layer_3_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_3_output_0 layer
Fingerprint after layer_3_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_3_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for training set: 100% 276/276 [00:00<00:00, 1408306.45it/s]
InV 0
impure:
[[24, 132, 31, 217, 91, 189, 104, 94], [0, 1, 0, 1, 1, 0, 0, 1], -1]

Fingerprinting VAL data after layer_3_output_0 layer
Fingerprint after layer_3_output_0. ((301, 256) inputs, (301, 256) neurons)
PRINTING ALL RULES.
impure:
[[24, 132, 31, 217, 91, 189, 104, 94], [0, 1, 0, 1, 1, 0, 0, 1], -1]


In [ ]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]


df_op.to_csv("./rules_3_1.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_3_0.csv",index=False)


****** RULES ********


In [ ]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_2_output_0' -type 3

CONFIG PARAMS: LAYER NAME: layer_2_output_0 ,TYPE: 3 ,INP TYPE: 0 ,ACTS: False ,Top/All: False
Layer Name: layer_2_output_0
Layers to be considered for fingerprinting: ['layer_2_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_2_output_0 layer
Fingerprint after layer_2_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_2_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for validation set: 100% 142/142 [00:00<00:00, 1167825.82it/s]
InV 0

Fingerprinting VAL data after layer_2_output_0 layer
Fingerprint after layer_2_output_0. ((301, 256) inputs, (301, 256) neurons)
PRINTING ALL RULES.


In [ ]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]


df_op.to_csv("./rules_2_1.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_2_0.csv",index=False)


****** RULES ********


In [ ]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_1_output_0' -type 3 -acts True

CONFIG PARAMS: LAYER NAME: layer_1_output_0 ,TYPE: 3 ,INP TYPE: 0 ,ACTS: True ,Top/All: False
Layer Name: layer_1_output_0
Layers to be considered for fingerprinting: ['layer_1_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_1_output_0 layer
Fingerprint after layer_1_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_1_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for training set: 100% 287/287 [00:00<00:00, 1523753.48it/s]
InV 0
impure:
[[152, 215, 19, 122, 170, 40, 213, 173, 219, 144, 114, 140, 222, 247, 253, 137], [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1], -1]

Fingerprinting VAL data after layer_1_output_0 layer
Fingerprint after layer_1_output_0. ((301, 256) inputs, (301, 256) neurons)
PRINTING ALL RULES.
impure:
[[152, 215, 19, 122, 170, 40, 213, 173, 219, 144, 114, 140, 222, 247, 253, 137

In [ ]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]


df_op.to_csv("./rules_1_1.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_1_0.csv",index=False)

****** RULES ********


In [ ]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_0_output_0' -type 3

CONFIG PARAMS: LAYER NAME: layer_0_output_0 ,TYPE: 3 ,INP TYPE: 0 ,ACTS: False ,Top/All: False
Layer Name: layer_0_output_0
Layers to be considered for fingerprinting: ['layer_0_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_0_output_0 layer
Fingerprint after layer_0_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_0_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for validation set: 100% 164/164 [00:00<00:00, 1230529.26it/s]
InV 0

Fingerprinting VAL data after layer_0_output_0 layer
Fingerprint after layer_0_output_0. ((301, 256) inputs, (301, 256) neurons)
PRINTING ALL RULES.


In [ ]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['train_recall'] > 50.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_0_1.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_0_0.csv",index=False)

****** RULES ********


In [ ]:

df_op = pd.read_csv("./rules_0_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 0 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_0_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 0 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_1_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 1 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_1_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 1 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_2_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 2 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_2_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 2 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_3_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 3 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_3_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 3 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_4_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 4 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_4_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 4 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_5_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 5 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_5_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 5 1:", train_f1,",",test_f1)



RULE 0 0: 9.655172413793103 , 15.602836879432624
RULE 0 1: 5.19159456118665 , 1.162790697674419
RULE 1 0: 7.88863109048724 , 8.823529411764707
RULE 1 1: 3.0000000000000004 , 2.312138728323699
RULE 2 0: 3.32541567695962 , 1.5267175572519085
RULE 2 1: 8.038976857490866 , 10.0
RULE 3 0: 1.9138755980861248 , 1.5267175572519085
RULE 3 1: 4.708798017348204 , 3.4482758620689653
RULE 4 0: 0.4819277108433735 , 1.5267175572519085
RULE 4 1: 4.950495049504951 , 1.162790697674419
RULE 5 0: 1.4388489208633093 , 1.5267175572519085
RULE 5 1: 14.806110458284373 , 1.162790697674419


In [ ]:
df_op = pd.read_csv("./rules_5_0.csv")
df_op = df_op[df_op.index == 0]
rule_5_0_neurons = df_op['neurons']
rule_5_0_signature = df_op['signature']


df_op = pd.read_csv("./rules_5_1.csv")
df_op = df_op[df_op.index == 0]
rule_5_1_neurons = df_op['neurons']
rule_5_1_signature = df_op['signature']

df_op = pd.read_csv("./rules_4_0.csv")
df_op = df_op[df_op.index == 0]
rule_4_0_neurons = df_op['neurons']
rule_4_0_signature = df_op['signature']


df_op = pd.read_csv("./rules_4_1.csv")
df_op = df_op[df_op.index == 0]
rule_4_1_neurons = df_op['neurons']
rule_4_1_signature = df_op['signature']


print(rule_4_0_neurons.array[0])
print(rule_4_0_signature.array[0])
print(rule_4_1_neurons.array[0])
print(rule_4_1_signature.array[0])


print(rule_5_0_neurons.array[0])
print(rule_5_0_signature.array[0])
print(rule_5_1_neurons.array[0])
print(rule_5_1_signature.array[0])

[29, 215, 181, 86, 192, 0, 70, 147, 91, 26, 250, 195]
['<=', 0.2756440043449402, '>', -0.5447605848312378, '>', -0.05165771581232548, '<=', 0.371839240193367, '<=', 0.3212895691394806, '>', -0.12406855449080467, '>', -0.15119200199842453, '<=', 0.07985635101795197, '>', 0.07166659459471703, '<=', 0.22430628538131714, '>', 0.11450915783643723, '>', 0.19882015138864517]
[29, 215, 181, 86, 192, 0, 70, 147, 144, 76, 193]
['<=', 0.2756440043449402, '>', -0.5447605848312378, '>', -0.05165771581232548, '<=', 0.371839240193367, '<=', 0.3212895691394806, '>', -0.12406855449080467, '>', -0.15119200199842453, '>', 0.07985635101795197, '>', 0.14457079768180847, '>', 0.44639018177986145, '<=', 0.4770570695400238]
[155, 191, 181, 138, 157, 103, 192, 204, 134, 169, 32]
[1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0]
[155, 97, 39, 219, 23, 11]
[0, 0, 0, 0, 1, 1]


In [ ]:
def check_pattern(layer_vals: list, neuron_ids: list, neuron_sig: list) -> bool:
    """
        Check if the provided layer values satisfy the provided neuron signature.
    :param layer_vals:
    :param neuron_ids:
    :param neuron_sig:
    :return:
    """
    found = True
    oper = -1
    # layer_vals = (layer_vals).flatten()

    for ind in range(0, len(neuron_sig)):
        if ind % 2 == 0:
            op = neuron_sig[ind]
            if op == '<=':
                oper = 0
            else:
                oper = 1
        else:
            v = int(neuron_ids[(int)(ind / 2)])
            vsig = float(neuron_sig[ind])
            val = float(layer_vals[v])
            # print(v,vsig,val,oper)
            if oper == 0:
                if val > vsig:
                    # print(v,val,vsig,oper)
                    found = False
                    break
            else:
                if val <= vsig:
                    # print(v,val,vsig,oper)
                    found = False
                    break
            oper = -1

    return found

def get_suffix_cluster(neuron_ids, neuron_sig, suffixes, VAL=False):
    # Get the cluster of inputs that such that all inputs in the cluster
    # have provided on/off signature for the provided neurons.
    #
    # The returned cluster is an array of indices (into mnist.train.images).
    if (VAL == False):
        return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

    matched_ids = []
    # print(len(suffixes))
    for indx in range(0, len(suffixes)):
        if (check_pattern(suffixes[indx], neuron_ids, neuron_sig) == True):
            matched_ids.append(indx)
    # print(matched_ids)
    return matched_ids



In [ ]:
import tensorflow as tf
import keras
model =tf.keras.models.load_model('/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5')
x_train = np.load('./x_train_npy.npy')
y_train = np.load('./y_train_npy.npy')

In [ ]:
from keras import backend

func5 = None
func4 = None

for layer in model.layers:
    print(layer.name)
    if (layer.name == 'layer_5_output_0'):
      func5 = backend.function(model.input, [layer.output])
    if (layer.name == 'layer_4_output_0'):
      func4 = backend.function(model.input, [layer.output])


fingerprint_5 = []
fingerprint_4 = []

if (func5 != None):
  fingerprint_5 = func5(x_train)

if (func4 != None):
  fingerprint_4 = func4(x_train)

layer_0_input_0
layer_0_output_0
layer_1_output_0
layer_2_output_0
layer_3_output_0
layer_4_output_0
layer_5_output_0
layer_6_output_0


In [ ]:
#print(len(x_train))
#x_train_flat = []
#for indx in range(0, len(x_train)):
#  x_train_flat.append((x_train[indx]).flatten())

#x_train_flat = np.array(x_train_flat)
print(np.shape(x_train))
#print(np.shape(x_train_flat))

print(np.shape(fingerprint_4[0]))
print(np.shape(fingerprint_5[0]))

(1202, 5)
(1202, 256)
(1202, 256)


In [ ]:
length = len(x_train[0])


x_train_min = np.zeros(length)
x_train_max = np.zeros(length)

for indx in range(0,length):
  x_train_min[indx] = np.min(x_train[:,indx])
  x_train_max[indx] = np.max(x_train[:,indx])

print(x_train_min)
print(x_train_max)



###########
rule_neurons_list_4_1 = []
rule_neurons = (rule_4_1_neurons.array[0]).split(",")
for indx in range(0, len(rule_neurons)):
    rule_neurons[indx] = (rule_neurons[indx]).strip()
    rule_neurons[indx] = (rule_neurons[indx]).replace("[", "")
    rule_neurons[indx] = (rule_neurons[indx]).replace("]","")
    rule_neurons_list_4_1.append(int(rule_neurons[indx]))

print(rule_neurons_list_4_1)

rule_sig_list_4_1 = []
rule_sig = (rule_4_1_signature.array[0]).split(",")
for indx in range(0, len(rule_sig)):
    rule_sig[indx] = (rule_sig[indx]).strip()
    rule_sig[indx] = (rule_sig[indx]).replace("[", "")
    rule_sig[indx] = (rule_sig[indx]).replace("]","")
    if (indx % 2 == 0):
      rule_sig[indx] = (rule_sig[indx]).replace("'", "")
      rule_sig_list_4_1.append(rule_sig[indx])
    else:
      rule_sig_list_4_1.append(float(rule_sig[indx]))

print(rule_sig_list_4_1)

fngprnt = fingerprint_4[0]
indices = get_suffix_cluster(rule_neurons_list_4_1, rule_sig_list_4_1, fngprnt,VAL=True)
print("indices:", len(indices))




x_train_4_1 = []
fngprnt_4_1 = []
inp_ex_4_1 = []
finger_ex_4_1 = []
for indx in range(0, len(indices)):
    if (indx == 0):
      inp_ex_4_1 = x_train[indices[indx]]
      finger_ex_4_1 = fingerprint_4[0][indices[indx]]

    x_train_4_1.append(x_train[indices[indx]])
    fngprnt_4_1.append(fingerprint_4[0][indices[indx]])

x_train_4_1= np.array(x_train_4_1)
fngprnt_4_1 = np.array(fngprnt_4_1)

print(np.shape(x_train_4_1))
x_train_min_4_1 = np.zeros(length)
x_train_max_4_1 = np.zeros(length)

for indx in range(0,length):
  x_train_min_4_1[indx] = np.min(x_train_4_1[:,indx])
  x_train_max_4_1[indx] = np.max(x_train_4_1[:,indx])

print(x_train_min_4_1)
print(x_train_max_4_1)

print(np.shape(fngprnt_4_1))
fngprnt_min_4_1 = np.zeros(len(fngprnt_4_1[0]))
fngprnt_max_4_1 = np.zeros(len(fngprnt_4_1[0]))

for indx in range(0,len(fngprnt_4_1[0])):
  fngprnt_min_4_1[indx] = np.min(fngprnt_4_1[:,indx])
  fngprnt_max_4_1[indx] = np.max(fngprnt_4_1[:,indx])

print(fngprnt_min_4_1)
print(fngprnt_max_4_1)


print("INP:", inp_ex_4_1)
print("FINGER:", finger_ex_4_1)

[-0.85240561 -1.1463989  -1.188622   -1.23082471 -0.81692582]
[5.4302597  2.1834619  1.79929733 1.31291699 3.59591293]
[29, 215, 181, 86, 192, 0, 70, 147, 144, 76, 193]
['<=', 0.2756440043449402, '>', -0.5447605848312378, '>', -0.05165771581232548, '<=', 0.371839240193367, '<=', 0.3212895691394806, '>', -0.12406855449080467, '>', -0.15119200199842453, '>', 0.07985635101795197, '>', 0.14457079768180847, '>', 0.44639018177986145, '<=', 0.4770570695400238]
indices: 20
(20, 5)
[-0.81591541 -1.1463989  -0.91699296 -1.23082471 -0.78562462]
[-0.2812542  -0.63931352  1.79929733 -0.72336125 -0.44460684]
(20, 256)
[ 1.58695996e-01  3.07415068e-01  6.77956492e-02 -3.45776826e-01
 -9.15562287e-02 -5.88296428e-02 -4.90033388e-01  7.03734532e-02
  3.58857304e-01  2.16475919e-01 -4.39988256e-01  3.02702397e-01
  7.96297640e-02  5.53577691e-02  1.70178771e-01 -8.19604024e-02
  2.85947267e-02  2.60811925e-01 -5.93821764e-01  5.02605885e-02
 -2.65997928e-03  7.55193532e-02  8.19150358e-03  3.51664126e-0

In [ ]:
length = len(x_train[0])


x_train_min = np.zeros(length)
x_train_max = np.zeros(length)

for indx in range(0,length):
  x_train_min[indx] = np.min(x_train[:,indx])
  x_train_max[indx] = np.max(x_train[:,indx])

print(x_train_min)
print(x_train_max)



###########
rule_neurons_list_5_1 = []
rule_neurons = (rule_5_1_neurons.array[0]).split(",")
for indx in range(0, len(rule_neurons)):
    rule_neurons[indx] = (rule_neurons[indx]).strip()
    rule_neurons[indx] = (rule_neurons[indx]).replace("[", "")
    rule_neurons[indx] = (rule_neurons[indx]).replace("]","")
    rule_neurons_list_5_1.append(int(rule_neurons[indx]))

print(rule_neurons_list_5_1)

rule_sig_list_5_1 = []
rule_sig = (rule_5_1_signature.array[0]).split(",")
for indx in range(0, len(rule_sig)):
    rule_sig[indx] = (rule_sig[indx]).strip()
    rule_sig[indx] = (rule_sig[indx]).replace("[", "")
    rule_sig[indx] = (rule_sig[indx]).replace("]","")
    rule_sig_list_5_1.append(int(rule_sig[indx]))

print(rule_sig_list_5_1)

fngprnt = (fingerprint_5[0] > 0.0).astype('int')
indices = get_suffix_cluster(rule_neurons_list_5_1, rule_sig_list_5_1, fngprnt)
print("indices:", len(indices))




x_train_5_1 = []
fngprnt_5_1 = []
inp_ex_5_1 = []
finger_ex_5_1 = []
for indx in range(0, len(indices)):
    if (indx == 0):
      inp_ex_5_1 = x_train[indices[indx]]
      finger_ex_5_1 = fingerprint_5[0][indices[indx]]

    x_train_5_1.append(x_train[indices[indx]])
    fngprnt_5_1.append(fingerprint_5[0][indices[indx]])

x_train_5_1= np.array(x_train_5_1)
fngprnt_5_1 = np.array(fngprnt_5_1)

print(np.shape(x_train_5_1))
x_train_min_5_1 = np.zeros(length)
x_train_max_5_1 = np.zeros(length)

for indx in range(0,length):
  x_train_min_5_1[indx] = np.min(x_train_5_1[:,indx])
  x_train_max_5_1[indx] = np.max(x_train_5_1[:,indx])

print(x_train_min_5_1)
print(x_train_max_5_1)

print(np.shape(fngprnt_5_1))
fngprnt_min_5_1 = np.zeros(len(fngprnt_5_1[0]))
fngprnt_max_5_1 = np.zeros(len(fngprnt_5_1[0]))

for indx in range(0,len(fngprnt_5_1[0])):
  fngprnt_min_5_1[indx] = np.min(fngprnt_5_1[:,indx])
  fngprnt_max_5_1[indx] = np.max(fngprnt_5_1[:,indx])

print(fngprnt_min_5_1)
print(fngprnt_max_5_1)


print("INP:", inp_ex_5_1)
print("FINGER:", finger_ex_5_1)

[-0.85240561 -1.1463989  -1.188622   -1.23082471 -0.81692582]
[5.4302597  2.1834619  1.79929733 1.31291699 3.59591293]
[155, 97, 39, 219, 23, 11]
[0, 0, 0, 0, 1, 1]
indices: 63
(63, 5)
[-0.40817672 -1.1463989   0.16952318 -1.23082471 -0.72578561]
[ 2.25719643 -0.47028506  1.79929733  1.31291699 -0.46875209]
(63, 256)
[1.03499733e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.83155972e-01 3.03180248e-01 0.00000000e+00 9.93944705e-05
 0.00000000e+00 0.00000000e+00 7.14892000e-02 0.00000000e+00
 1.08241625e-02 0.00000000e+00 0.00000000e+00 4.21178080e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.52980585e-02
 0.00000000e+00 0.00000000e+00 2.50227273e-01 0.00000000e+00
 0.00000000e+00 9.90967005e-02 0.00000000e+00 3.38076949e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.00563449e-01
 0.00000000e+00 0.00000000e+00 1.05809011e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.68780538e-02
 0.0000000

**PROOFS WITH MARABOU**

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Marabou_bld')
!pwd
!ls -lt

/content/drive/MyDrive/Marabou_bld
total 54578
-rw------- 1 root root   237800 Dec 27 01:00 cnn_max_mnist2.h5
-rw------- 1 root root  7840128 Dec 27 01:00 x_test_npy.npy
-rw------- 1 root root 47040128 Dec 27 01:00 x_train_npy.npy
-rw------- 1 root root    10128 Dec 27 01:00 y_test_npy.npy
-rw------- 1 root root    60128 Dec 27 01:00 y_train_npy.npy
-rw------- 1 root root     7194 Dec 16 18:56 evaluateWithMarabou.log
drwx------ 2 root root     4096 Dec 12 17:57 ProphecyPlus
-rw------- 1 root root   305212 Nov 25 05:34 mnist_cnn.onnx
-rw------- 1 root root   305212 Nov 25 05:33 REDPropiaFinal.onnx
drwx------ 2 root root     4096 Nov 25 05:33 tmp_model
drwx------ 2 root root     4096 Nov 24 21:00 tools
drwx------ 2 root root     4096 Nov 24 20:59 build
drwx------ 2 root root     4096 Nov 24 20:59 resources
drwx------ 2 root root     4096 Nov 24 20:37 maraboupy
-rw------- 1 root root     2092 Nov 24 20:06 AUTHORS
-rw------- 1 root root     2274 Nov 24 20:06 CHANGELOG.md
-rw------- 1 root 

In [ ]:
path = os.environ['PATH']
print(path)
#os.environ['PATH'] = '/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/Marabou/build/Marabou:/content/Marabou/build/Marabou/build:/content/Marabou/build/Marabou/build/bin:/content/Marabou/build/Marabou:/content/Marabou/build/Marabou/build:/content/Marabou/build/Marabou/build/bin'
print(os.environ['PATH'])
os.environ['PATH'] = path + ':/content/drive/MyDrive/Marabou_bld:/content/drive/MyDrive/Marabou_bld/build:/content/drive/MyDrive/Marabou_bld/build/bin'
print(os.environ['PATH'])

/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/drive/MyDrive/Marabou_bld:/content/drive/MyDrive/Marabou_bld/build:/content/drive/MyDrive/Marabou_bld/build/bin


**PROVE RULE FOR ROBUSTNESS OF OP**

In [ ]:
import os
import numpy as np
from maraboupy import Marabou
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *

options = Marabou.createOptions(verbosity = 1, numWorkers=1, numBlasThreads=1,snc=True)

filename = "/content/drive/MyDrive/FAA_CaseStudy/renamed_model.onnx"
network_a = Marabou.read_onnx(filename)


print("INPUT VARS")
invars = network_a.inputVars[0][0].flatten()



for indx in range(0,len(invars)):
    i = invars[indx]
    v = Var(i)
    network_a.setLowerBound(i,x_train_min_4_1[i])
    network_a.setUpperBound(i,x_train_max_4_1[i])
    #network_a.setLowerBound(i,inp_ex_4_1[indx])
    #network_a.setUpperBound(i,inp_ex_4_1[indx])


print("LAYER VARS MAP")
print(network_a.layerNameToVariables)

dense_4_neurons = network_a.layerNameToVariables["layer_4_output_0"][0]
print(np.shape(dense_4_neurons))
print(len(dense_4_neurons))





for indx in range(0, len(dense_4_neurons)):
    neuron_indx = dense_4_neurons[indx] - dense_4_neurons[0]
    network_a.setLowerBound(dense_4_neurons[indx], fngprnt_min_4_1[neuron_indx])
    network_a.setUpperBound(dense_4_neurons[indx], fngprnt_max_4_1[neuron_indx])
    #network_a.setLowerBound(dense_4_neurons[indx], finger_ex_4_1[neuron_indx] - 0.5 )
    #network_a.setUpperBound(dense_4_neurons[indx], finger_ex_4_1[neuron_indx] + 0.5 )

    if (neuron_indx in rule_neurons_list_4_1):
      v = Var(dense_4_neurons[indx])

      for indx in range(0, len(rule_neurons_list_4_1)):
        if (rule_neurons_list_4_1[indx] == neuron_indx):
          index = indx
          sig_indx = (2*index) + 1
          print("N:", neuron_indx, index)
          print("OP:", rule_sig_list_4_1[sig_indx-1])
          print("VAL:", rule_sig_list_4_1[sig_indx])
         # if (rule_sig_list_4_1[sig_indx-1] == '<='):
         #   network_a.addConstraint(float(rule_sig_list_4_1[sig_indx]) >= v)
         # else:
         #   network_a.addConstraint(v >= float(rule_sig_list_4_1[sig_indx]))



print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
print(outvars)
label_var = Var(outvars[0])
# |op| >= OP_X + 0.19200696 should be UNSAT
#(-0.15859113982634868 - 0.19200696) <= OP <= (-0.15859113982634868 + 0.19200696)
# CONSTRAINT 1
network_a.addConstraint(label_var >= (0.2 + -0.15859113982634868))

# CONSTRAINT 2
#network_a.addConstraint((-0.19200696 + -0.15859113982634868) >= label_var)





INPUT VARS
LAYER VARS MAP
{'layer_0_input_0': array([[0, 1, 2, 3, 4]]), 'layer_0_output_0': array([[  5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
         18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
         31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
         44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
         57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
         83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
         96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
        109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
        122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
        135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
        148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
        161, 162, 163, 164, 165,

In [ ]:
#print(network_a.getInputQuery())
sat_unsat,vals, stats = network_a.solve(options = options)

print("sat_unsat:", sat_unsat)
print("vals:", vals)
print("stats:", stats)


sat
input 0 = -0.2812548182892106
input 1 = -1.146398901939392
input 2 = -0.916992859677764
input 3 = -1.2308247089385986
input 4 = -0.785624623298645
output 0 = 1.210201327984187
sat_unsat: sat
vals: {0: -0.2812548182892106, 1: -1.146398901939392, 2: -0.916992859677764, 3: -1.2308247089385986, 4: -0.785624623298645, 5: 0.35211024161175136, 6: -0.243502789936913, 7: 0.1831146108695369, 8: -0.6475721531250953, 9: 1.0281109293753126, 10: -0.07479665484421355, 11: 0.3654850064524065, 12: -0.35810033865097357, 13: -0.0106931621604541, 14: 0.3131653263426629, 15: 0.1956856006736597, 16: -0.10375635485787021, 17: -0.511471247275848, 18: 0.6220272353853596, 19: -0.32928674362726584, 20: 0.35747948003016405, 21: -0.004419812058303954, 22: 0.18083502627821432, 23: 0.7972320467129479, 24: -0.09725189172460878, 25: -0.22420457105205405, 26: -0.9118781713665579, 27: -0.2624909543700713, 28: 0.09284481315527893, 29: -0.13310001887944856, 30: 0.2960197005310702, 31: -1.1624097346319564, 32: 0.371764